In [ ]:
!pip install plotly --upgrade --quiet
!pip install openpyxl --upgrade --quiet

In [ ]:
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
os.listdir("../input")

In [ ]:
volve_df = pd.read_excel('../input/volve-production-data/Volve production data.xlsx', sheet_name='Monthly Production Data')

In [ ]:
volve_df

In [ ]:
volve_df.info()

In [ ]:
# Number of NaN values for each series
volve_df.isna().sum()

In [ ]:
volve_df['Wellbore name'].unique()

In [ ]:
# Dropping the first row
volve_df.drop([0], inplace=True)

In [ ]:
# Resetting the indices
volve_df.reset_index(drop=True, inplace=True)

In [ ]:
volve_df

In [ ]:
volve_df.fillna(0, inplace=True)

In [ ]:
# All the NaN values were replaced by zero (0)
volve_df.isna().sum()

In [ ]:
volve_df.head()

In [ ]:
volve_df.sample(5)

In [ ]:
# Display the different data types
volve_df.dtypes

In [ ]:
volve_df[['NPDCode', 'Year', 'Month']] = volve_df[['NPDCode', 'Year', 'Month']].astype(int)

In [ ]:
volve_df[['GI']] = volve_df[['GI']].astype(float)

In [ ]:
# Convert the Month column from int to month_abbr (Ex: 1 -> Jan, 2-> Feb, ...etc)
import calendar

abbreviations = dict(enumerate(calendar.month_abbr))
volve_df['Month'] = volve_df['Month'].map(abbreviations)

In [ ]:
volve_df['Month'] = pd.Categorical(volve_df.Month, 
                             categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                             ordered=True)

In [ ]:
volve_df.head()

In [ ]:
# All the conversions were successful
volve_df.dtypes

# Analysis and visualization

Now let's analyze our data and visualize the production using various plots.

In [ ]:
# Calculating the total Oil, Gas, and Water produced
total_oil = volve_df.Oil.sum()
total_gas = volve_df.Gas.sum()
total_water = volve_df.Water.sum()

total_production = total_oil + total_gas + total_water

In [ ]:
print(f"The total oil produced is {total_oil} cubic meters.")
print(f"The total gas produced is {total_gas} cubic meters.")
print(f"The total water produced is {total_water} cubic meters.")

In [ ]:
values = [total_oil, total_gas, total_water]
labels = ['Oil', 'Gas', 'Water']


fig = px.pie(values=values, names=labels, hole=.4, labels=labels, color=labels,
             color_discrete_map={'Oil':'lime', 'Gas':'red', 'Water':'blue'},
             title="<b>Fluid production fraction</b>")

fig.update_traces(hovertemplate="<br>".join(["Fluid: %{label}", "Production: %{value} Sm<sup>3</sup>"]))

fig.update_layout(font_size=14)

fig.show()

In [ ]:
sorted_df = volve_df.sort_values(['Year', 'Month'])

In [ ]:
# Here, I prefer to reset the indices but it is your choice to do it or not
sorted_df.reset_index(drop=True, inplace=True)

In [ ]:
sorted_df

In [ ]:
# Adding new columns to compute the cumulative Oil, Gas, and Water produced
sorted_df['cum_oil'] = sorted_df.Oil.cumsum()
sorted_df['cum_gas'] = sorted_df.Gas.cumsum()
sorted_df['cum_water'] = sorted_df.Water.cumsum()

sorted_df['total_production'] = sorted_df['cum_oil'] + sorted_df['cum_gas'] + sorted_df['cum_water']

In [ ]:
sorted_df

In [ ]:
# Create a data frame that only contains the data related to hydrocarbon (Oil & Gas) and water production per year
annual_df = sorted_df.groupby('Year')[['Oil', 'Gas', 'Water']].sum()

In [ ]:
annual_df

In [ ]:
annual_df['cum_oil'] = sorted_df.groupby('Year')[['Oil']].sum().cumsum()
annual_df['cum_gas'] = sorted_df.groupby('Year')[['Gas']].sum().cumsum()
annual_df['cum_water'] = sorted_df.groupby('Year')[['Water']].sum().cumsum()

# Note: Another and probably simpler alternative to do this would be to use the newly created data frame with the syntax
# annual_df['cum_oil'] = annual_df.Oil.cumsum()

In [ ]:
annual_df

In [ ]:
layout = go.Layout(title='<b>Annual Oil Production (Sm<sup>3</sup>)</b>',
                   xaxis_title='Year',
                   yaxis_title='Oil Production',
                   height=580,
                   width=900,
                   font_size=14)


fig = go.Figure(layout=layout)

fig.add_traces(data=go.Bar(x=annual_df.index, y=annual_df.Oil,
                           marker=dict(color='lime', line=dict(color='white', width=1.2)),
                           opacity=.9,
                           hovertemplate='<br>'.join(['Year = %{x}', 'Oil = %{y}']),
                           name='Distribution'))

fig.add_traces(data=go.Scatter(x=annual_df.index, y=annual_df.Oil,
                               line=dict(dash='dash', color='red', width=3),
                               marker_size=10,
                               hovertemplate='<br>'.join(['Year = %{x}', 'Oil = %{y}']),
                               name='Profile'))

fig.show()

In [ ]:
layout = go.Layout(title='<b>Annual Gas Production (Sm<sup>3</sup>)</b>',
                   xaxis_title='Year',
                   yaxis_title='Gas Production',
                   height=580,
                   width=900,
                   font_size=14)


fig = go.Figure(layout=layout)

fig.add_traces(data=go.Bar(x=annual_df.index, y=annual_df.Gas,
                           marker=dict(color='tomato', line=dict(color='white', width=1.2)),
                           hovertemplate='<br>'.join(['Year = %{x}', 'Gas = %{y}']),
                           name='Distribution'))

fig.add_traces(data=go.Scatter(x=annual_df.index, y=annual_df.Gas,
                               line=dict(dash='dash', color='darkcyan', width=3),
                               marker_size=10,
                               hovertemplate='<br>'.join(['Year = %{x}', 'Gas = %{y}']),
                               name='Profile'))

fig.show()

In [ ]:
layout = go.Layout(title='<b>Cum Oil Vs. Cum Water Production (Sm<sup>3</sup>)</b>',
                   xaxis_title='Year',
                   yaxis_title='Cumulative Production',
                   hovermode='x unified',
                   height=580,
                   width=900,
                   font_size=14)


fig = go.Figure(layout=layout)

fig.add_traces(data=go.Scatter(x=annual_df.index, y=annual_df.cum_oil,
                               line=dict(color='green', width=3),
                               marker_size=10,
                               name='Cum Oil'))

fig.add_traces(data=go.Scatter(x=annual_df.index, y=annual_df.cum_water,
                               line=dict(dash='dash', color='blue', width=3),
                               marker=dict(symbol='square', size=10),
                               name='Cum Water'))

fig.show()

In [ ]:
layout = go.Layout(title='<b>Oil Vs. Water Production (Sm<sup>3</sup>)</b>',
                   xaxis_title='Year',
                   yaxis_title='Total Production',
                   height=580,
                   width=900,
                   hovermode='x unified',
                   font_size=14)


fig = go.Figure(layout=layout)

fig.add_traces(data=go.Scatter(x=annual_df.index, y=annual_df.Oil,
                               line=dict(color='lime', width=3),
                               marker_size=10,
                               name='Oil'))

fig.add_traces(data=go.Scatter(x=annual_df.index, y=annual_df.Water,
                               line=dict(dash='dash', color='blue', width=3),
                               marker=dict(symbol='square', size=10),
                               name='Water'))

fig.show()

In [ ]:
# Create a data frame to account for the monthly Oil production
monthly_oil_df = sorted_df.pivot_table('Oil', 'Month', 'Year')
monthly_oil_df

In [ ]:
# Let's replace these NaN values by zero (0)
monthly_oil_df.fillna(0, inplace=True)

In [ ]:
fig = px.imshow(monthly_oil_df,
                color_continuous_scale='Greens',
                labels=dict(x='Year', y='Month'),
                title='<b>Monthly Oil Production</b>',
                height=610, width=900)

fig.update_traces(hovertemplate="<br>".join(['Year: %{x}', 'Month:  %{y}', 'Produced Oil (Sm<sup>3</sup>): %{z}']), name='')

fig.update_layout(font_size=14)

In [ ]:
monthly_gas_df = sorted_df.pivot_table('Gas', 'Month', 'Year')
monthly_gas_df

In [ ]:
monthly_gas_df.fillna(0, inplace=True)

In [ ]:
fig = px.imshow(monthly_gas_df,
                color_continuous_scale='Reds',
                labels=dict(x='Year', y='Month'),
                title='<b>Monthly Gas Production</b>',
                height=610, width=900)

fig.update_traces(hovertemplate="<br>".join(['Year: %{x}', 'Month:  %{y}', 'Produced Gas (Sm<sup>3</sup>): %{z}']), name='')

fig.update_layout(font_size=14)

fig.show()

In [ ]:
sns.set_style('darkgrid')

In [ ]:
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
# Plotting all the graphs in a single grid

fig, axes = plt.subplots(2, 3, figsize=(18, 9))

# Annual Oil production
axes[0, 0].bar(annual_df.index, annual_df.Oil, color='lime')
axes[0, 0].plot(annual_df.index, annual_df.Oil, marker='o', ms=8, ls='--', lw=2, c='red')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Oil production')
axes[0, 0].set_title('Annual Oil Production')

# Monthly Oil production
axes[0, 1].set_title('Monthly Oil production')
sns.heatmap(monthly_oil_df, cmap='Greens', ax=axes[0, 1])

# Cum Oil Vs. Cum Water
axes[0, 2].plot(annual_df.index, annual_df.cum_oil, 'o-g')
axes[0, 2].plot(annual_df.index, annual_df.cum_water, 's--b')
axes[0, 2].set_xlabel('Year')
axes[0, 2].set_ylabel('Cumulative production')
axes[0, 2].set_title('Cum Oil Vs. Cum Water production')
axes[0, 2].legend(['Oil', 'Water'])

# Annual Gas production
axes[1, 0].bar(annual_df.index, annual_df.Gas, color='tomato')
axes[1, 0].plot(annual_df.index, annual_df.Gas, marker='o', ms=8, ls='--', lw=2, c='darkcyan')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Gas production')
axes[1, 0].set_title('Annual Gas Production')

# Monthly Gas production
axes[1, 1].set_title('Monthly Gas production')
sns.heatmap(monthly_gas_df, cmap='Reds', ax=axes[1, 1])

# Oil production Vs. Water production
axes[1, 2].plot(annual_df.index, annual_df.Oil, marker='o', ms=8, ls='-', lw=2, c='lime')
axes[1, 2].plot(annual_df.index, annual_df.Water, marker='s', ms=8, ls='--', lw=2, c='blue')
axes[1, 2].set_xlabel('Year')
axes[1, 2].set_ylabel('Total production')
axes[1, 2].set_title('Oil Vs. Water production')
axes[1, 2].legend(['Oil', 'Water'])


plt.tight_layout(pad=2)

In [ ]:
wellbores_df = volve_df.groupby(['Wellbore name'], as_index=False)[['On Stream', 'Oil', 'Gas', 'Water', 'GI', 'WI']].sum()

In [ ]:
wellbores_df

In [ ]:
fig = px.bar(wellbores_df, x='Wellbore name', y='Oil',
             color='Wellbore name',
             color_discrete_sequence=px.colors.sequential.Aggrnyl)

fig.update_traces(marker_line_color='white', marker_line_width=1.2, showlegend=False)

fig.update_layout(title='<b>Total Oil Production per Wellbore (Sm<sup>3</sup>)</b>',
                  height=580, width=900,
                  font_size=14)

fig.show()

In [ ]:
fig = px.bar(wellbores_df, x='Wellbore name', y='Gas',
             color='Wellbore name',
             color_discrete_sequence=px.colors.sequential.Sunsetdark_r)

fig.update_traces(marker_line_color='white', marker_line_width=1.2, showlegend=False)

fig.update_layout(title='<b>Total Gas Production per Wellbore (Sm<sup>3</sup>)</b>',
                  height=580, width=900,
                  font_size=14)

fig.show()

In [ ]:
fig = px.bar(wellbores_df, x='Wellbore name', y='On Stream',
             color='Wellbore name',
             title='<b>Total Operating time (hrs)</b>',
             color_discrete_sequence=px.colors.qualitative.D3)

fig.update_traces(marker_line_color='white', marker_line_width=1.2, showlegend=False)

fig.update_layout(font_size=14, height=580, width=900)

fig.show()

In [ ]:
# Creating a data frame for injection wells
injection_wells_df = wellbores_df.loc[(wellbores_df.WI > 0) | (wellbores_df.GI > 0)].reset_index(drop=True)

In [ ]:
fig = px.bar(injection_wells_df, x='Wellbore name', y='WI',
             color_discrete_sequence=['deepskyblue'],
             labels=dict(WI='Injected Water'))

fig.update_traces(marker_line_color='white', marker_line_width=1.2)

fig.update_layout(title='<b>Total injected water (Sm<sup>3</sup>)</b>',
                  height=580, width=900,
                  font_size=14)

fig.show()

In [ ]:
# Creating a data frame for production wells
production_wells_df = wellbores_df.loc[(wellbores_df.Oil > 0) | (wellbores_df.Gas > 0) | (wellbores_df.Water > 0)].reset_index(drop=True)

In [ ]:
production_wells_df

In [ ]:
fig = px.bar(production_wells_df, x='Wellbore name', y='Oil',
             color='Wellbore name',
             color_discrete_sequence=px.colors.sequential.Aggrnyl)

fig.update_traces(marker_line_color='white', marker_line_width=1.2, showlegend=False)

fig.update_layout(title='<b>Total Oil Production per Wellbore (Sm<sup>3</sup>)</b>',
                  height=580, width=900,
                  font_size=14)

fig.show()

In [ ]:
fig = px.bar(production_wells_df, x='Wellbore name', y='Gas',
             color='Wellbore name',
             color_discrete_sequence=px.colors.sequential.Sunsetdark_r)

fig.update_traces(marker_line_color='white', marker_line_width=1.2, showlegend=False)

fig.update_layout(title='<b>Total Gas Production per Wellbore (Sm<sup>3</sup>)</b>',
                  height=580, width=900,
                  font_size=14)

fig.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 10))


axes[0, 0].set_title('Total Oil production per Wellbore (Sm$^3$)')
sns.barplot(x='Wellbore name', y='Oil', palette='viridis_r', data=production_wells_df, ax=axes[0, 0])


axes[0, 1].set_title('Total Gas production per Wellbore (Sm$^3$)')
sns.barplot(x='Wellbore name', y='Gas', palette='rocket_r', data=production_wells_df, ax=axes[0, 1])


axes[1, 0].set_title('Total Injected Water (Sm$^3$)')
axes[1, 0].set_xlabel('Wellbore name')
axes[1, 0].set_ylabel('Injected water')
axes[1, 0].bar(x='Wellbore name', height='WI', color='deepskyblue', data=injection_wells_df)


axes[1, 1].set_title('Total Operating time (hrs)')
sns.barplot(x='Wellbore name', y='On Stream', data=wellbores_df, palette='bright', ax=axes[1, 1])


plt.tight_layout(pad=2)

# Individual Wellbore production analysis

Now let's say we want to analyze a wellbore in particular. Maybe we want to know its oil production, gas production, operating time, ...etc throughout the years. We will have to create a new data frame to store the data of that particular wellbore.

Suppose we want to analyze the wellbore `15/9-F-5` for example.

In [ ]:
# Creating a data frame to store all data about the wellbore 15/9-F-5
well_f5_df = sorted_df[sorted_df['Wellbore name'] == '15/9-F-5'].drop(columns=['cum_oil', 'cum_gas', 'cum_water', 'total_production'])

In [ ]:
# Resetting the indices
well_f5_df.reset_index(drop=True, inplace=True)

In [ ]:
well_f5_df.head()

In [ ]:
well_f5_annual_df = well_f5_df.groupby('Year')[['On Stream', 'Oil', 'Gas', 'Water', 'GI', 'WI']].sum()

In [ ]:
well_f5_annual_df

In [ ]:
fig = px.bar(well_f5_annual_df,
             x=well_f5_annual_df.index,
             y=well_f5_annual_df.Oil,
             color_discrete_sequence=['mediumspringgreen'],
             height=580, width=900,
             title="<b>Annual Oil Production (Sm<sup>3</sup>)</b>")

fig.update_traces(marker_line_color='white', marker_line_width=1.2)

fig.update_layout(font_size=14)

fig.show()

In [ ]:
fig = px.bar(well_f5_annual_df,
             x=well_f5_annual_df.index,
             y=well_f5_annual_df.Gas,
             color_discrete_sequence=['crimson'],
             height=580, width=900,
             title="<b>Annual Gas Production (Sm<sup>3</sup>)</b>")

fig.update_traces(marker_line_color='white', marker_line_width=1.2)

fig.update_layout(font_size=14)

fig.show()

In [ ]:
fig = px.line(well_f5_annual_df, x=well_f5_annual_df.index, y=well_f5_annual_df['On Stream'],
              color_discrete_sequence=['blue'], markers=True,
              title='<b>Total Operating time per Year (hrs)</b>')

fig.update_layout(font_size=14, height=580, width=900)

fig.show()

In [ ]:
fig = px.bar(well_f5_annual_df, x=well_f5_annual_df.index, y=well_f5_annual_df.WI,
             color_discrete_sequence=['deepskyblue'],
             labels={'WI': 'Water Injection'},
             title='<b>Annual Water Injection (Sm<sup>3</sup>)</b>')

fig.update_traces(marker_line_color='white', marker_line_width=1.2)

fig.update_layout(font_size=14, height=580, width=900)

fig.show()

In [ ]:
# Creating a data frame to account for the monthly Oil production
well_f5_monthly_oil_df = well_f5_df.pivot_table('Oil', 'Month', 'Year')

In [ ]:
well_f5_monthly_oil_df

In [ ]:
# Replacing the NaN values by zero (0)
well_f5_monthly_oil_df.fillna(0, inplace=True)

In [ ]:
well_f5_monthly_oil_df

In [ ]:
fig = px.imshow(well_f5_monthly_oil_df,
                color_continuous_scale='Greens',
                labels=dict(x='Year', y='Month'),
                title='<b>Monthly Oil Production</b>')

fig.update_traces(hovertemplate="<br>".join(['Year: %{x}', 'Month:  %{y}', 'Produced Oil (Sm<sup>3</sup>): %{z}']), name='')

fig.update_layout(font_size=14, height=610, width=900)

In [ ]:
# Creating a data frame to account for the monthly Gas production
well_f5_monthly_gas_df = well_f5_df.pivot_table('Gas', 'Month', 'Year')

In [ ]:
well_f5_monthly_gas_df

In [ ]:
well_f5_monthly_gas_df.fillna(0, inplace=True)

In [ ]:
well_f5_monthly_gas_df

In [ ]:
fig = px.imshow(well_f5_monthly_gas_df,
                color_continuous_scale='Reds',
                labels=dict(x='Year', y='Month'),
                title='<b>Monthly Gas Production</b>')

fig.update_traces(hovertemplate="<br>".join(['Year: %{x}', 'Month:  %{y}', 'Produced Gas (Sm<sup>3</sup>): %{z}']), name='')

fig.update_layout(font_size=14, height=610, width=900)

In [ ]:
well_f5_monthly_WI_df = well_f5_df.pivot_table('WI', 'Month', 'Year')

In [ ]:
well_f5_monthly_WI_df.fillna(0, inplace=True)

In [ ]:
well_f5_monthly_WI_df

In [ ]:
fig = px.imshow(well_f5_monthly_WI_df,
                color_continuous_scale='Blues',
                labels=dict(x='Year', y='Month'),
                title='<b>Monthly Water Injection</b>')

fig.update_traces(hovertemplate="<br>".join(['Year: %{x}', 'Month:  %{y}', 'Injected Water (Sm<sup>3</sup>): %{z}']), name='')

fig.update_layout(font_size=14, height=610, width=900)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 9))

# Annual Oil production
axes[0, 0].bar(well_f5_annual_df.index, well_f5_annual_df.Oil, color='mediumspringgreen')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Oil production')
axes[0, 0].set_title('Annual Oil Production (Sm$^3$)')

# Annual Gas production
axes[0, 1].bar(well_f5_annual_df.index, well_f5_annual_df.Gas, color='crimson')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Gas production')
axes[0, 1].set_title('Annual Gas Production (Sm$^3$)')

# Annual Operating time
axes[0, 2].plot(well_f5_annual_df.index, well_f5_annual_df['On Stream'], 'o-b')
axes[0, 2].set_xlabel('Year')
axes[0, 2].set_ylabel('On Stream')
axes[0, 2].set_title('Total Operating time per Year (hrs)')

# Monthly Oil production
axes[1, 0].set_title('Monthly Oil production')
sns.heatmap(well_f5_monthly_oil_df, cmap='Greens', ax=axes[1, 0])

# Monthly Gas production
axes[1, 1].set_title('Monthly Gas production')
sns.heatmap(well_f5_monthly_gas_df, cmap='Reds', ax=axes[1, 1])

# Annual Water Injection
axes[1, 2].bar(well_f5_annual_df.index, well_f5_annual_df.WI, color='deepskyblue')
axes[1, 2].set_xlabel('Year')
axes[1, 2].set_ylabel('Injected Water')
axes[1, 2].set_title('Annual Water Injection (Sm$^3$)')


plt.tight_layout(pad=2)

In [ ]:
well_f12_df = sorted_df[sorted_df['Wellbore name'] == '15/9-F-12'].drop(columns=['cum_oil', 'cum_gas', 'cum_water', 'total_production'])

In [ ]:
well_f12_df.reset_index(drop=True, inplace=True)

In [ ]:
well_f12_df.head()

In [ ]:
well_f12_annual_df = well_f12_df.groupby('Year')[['On Stream', 'Oil', 'Gas', 'Water', 'GI', 'WI']].sum()

In [ ]:
well_f12_annual_df

In [ ]:
fig = px.bar(well_f12_annual_df,
             x=well_f12_annual_df.index,
             y=well_f12_annual_df.Oil,
             color_discrete_sequence=['mediumspringgreen'],
             height=580, width=900,
             title="<b>Annual Oil Production (Sm<sup>3</sup>)</b>")

fig.update_traces(marker_line_color='white', marker_line_width=1.2)

fig.update_layout(font_size=14)

fig.show()

In [ ]:
fig = px.bar(well_f12_annual_df,
             x=well_f12_annual_df.index,
             y=well_f12_annual_df.Gas,
             color_discrete_sequence=['crimson'],
             height=580, width=900,
             title="<b>Annual Gas Production (Sm<sup>3</sup>)</b>")

fig.update_traces(marker_line_color='white', marker_line_width=1.2)

fig.update_layout(font_size=14)

fig.show()

In [ ]:
well_f12_monthly_oil_df = well_f12_df.pivot_table('Oil', 'Month', 'Year')

In [ ]:
well_f12_monthly_oil_df

In [ ]:
well_f12_monthly_oil_df.fillna(0, inplace=True)

In [ ]:
fig = px.imshow(well_f12_monthly_oil_df,
                color_continuous_scale='Greens',
                labels=dict(x='Year', y='Month'),
                title='<b>Monthly Oil Production</b>')

fig.update_traces(hovertemplate="<br>".join(['Year: %{x}', 'Month:  %{y}', 'Produced Oil (Sm<sup>3</sup>): %{z}']), name='')

fig.update_layout(font_size=14, height=610, width=900)

In [ ]:
well_f12_monthly_gas_df = well_f12_df.pivot_table('Gas', 'Month', 'Year')

In [ ]:
well_f12_monthly_gas_df

In [ ]:
well_f12_monthly_gas_df.fillna(0, inplace=True)

In [ ]:
fig = px.imshow(well_f12_monthly_gas_df,
                color_continuous_scale='Reds',
                labels=dict(x='Year', y='Month'),
                title='<b>Monthly Gas Production</b>')

fig.update_traces(hovertemplate="<br>".join(['Year: %{x}', 'Month:  %{y}', 'Produced Gas (Sm<sup>3</sup>): %{z}']), name='')

fig.update_layout(font_size=14, height=610, width=900)

fig.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))


axes[0, 0].bar(well_f12_annual_df.index, well_f12_annual_df.Oil, color='mediumspringgreen')
axes[0, 0].set_title('Annual Oil Production (Sm$^3$)')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Oil production')

axes[0, 1].bar(well_f12_annual_df.index, well_f12_annual_df.Gas, color='crimson')
axes[0, 1].set_title('Annual Oil Production (Sm$^3$)')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Gas production')

axes[1, 0].set_title('Monthly Oil Production')
sns.heatmap(well_f12_monthly_oil_df, cmap='Greens', ax=axes[1, 0])

axes[1, 1].set_title('Monthly Gas Production')
sns.heatmap(well_f12_monthly_gas_df, cmap='Reds', ax=axes[1, 1])


plt.tight_layout(pad=2)

In [ ]:
def cleanse_data(data):
    """
    Performs Cleaning and Transformation of any dataset in the format of the Volve field's "Monthly Production Data".


        Arguments
       -----------
        data (pd.DataFrame) : Production dataset. The passed data frame should be in the appropriate format.

        Returns
       -----------
        df (pd.DataFrame) : Cleaned copy of the passed dataset.

        Sample Link
       -------------
        www.kaggle.com/dataset/5c4a19ebeb017a77adadce8c460e2a8ca1e6e84719f1022ba317328be9bdd267

    """
    import pandas as pd
    import calendar
    import sys


    if not isinstance(data, pd.DataFrame):
        sys.exit('Invalid input, please enter a dataframe.') # Still get an error I don't want here
    

    # Verify that the format is respected
    columns = ['Wellbore name', 'NPDCode', 'Year', 'Month', 'On Stream', 'Oil', 'Gas', 'Water', 'GI', 'WI']
    for a, b in zip(columns, data.columns):
        if a != b:
            sys.exit('Please make sure the entered dataset respects the default format.')

    # Create a copy to avoid modifying the original dataset
    df = data.copy()
    
    # Data cleaning
    df.drop([0], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.fillna(0, inplace=True)

    # Data transformation
    df[['NPDCode', 'Year', 'Month']] = df[['NPDCode', 'Year', 'Month']].astype(int)
    df[['GI']] = df[['GI']].astype(float)

    # Change numbers into month abbreviation
    abbrev = dict(enumerate(calendar.month_abbr))
    df['Month'] = df['Month'].map(abbrev)

    # Specify a custom order in which you want the months to be sorted. Otherwise they will be sorted by alphabetic order.
    df['Month'] = pd.Categorical(df.Month,
                                 categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                                 ordered=True)

    return df

In [ ]:
def create_sorted_df(data):
    """
    Creates a sorted data frame from any dataset in the format of the Volve field's "Monthly Production Data".


        Arguments
       -----------
        data (pd.DataFrame) : Data frame to be sorted. The passed data frame should be in the appropriate format.

        Returns
       -----------
        df (pd.DataFrame) : The result data frame is sorted by Year, then by Month. Three (03) columns are also added
                    to account for the cumulated Oil, Gas, and Water production respectively.

        Sample Link
       -------------
        www.kaggle.com/dataset/5c4a19ebeb017a77adadce8c460e2a8ca1e6e84719f1022ba317328be9bdd267

    """
    import pandas as pd


    # Clean and transform the data
    data = cleanse_data(data)

    # Sort the dataframe by Year, then by Month
    df = data.sort_values(['Year', 'Month'])
    df.reset_index(drop=True, inplace=True)

    # Create additional columns for the cumulated production of each fluid
    df['cum_oil'] = df.Oil.cumsum()
    df['cum_gas'] = df.Gas.cumsum()
    df['cum_water'] = df.Water.cumsum()

    return df

In [ ]:
def create_well_df(data, well_name):
    """
    Creates a data frame to store the data of a particular wellbore.


        Arguments
       -----------
        data (pd.DataFrame) : Production dataset. The passed data frame should be in the appropriate format.

        well_name (str) : Specifies the name of the wellbore you want to create a dataframe of.

        Returns
       -----------
        df (pd.DataFrame) : Returns a data frame that contains all the production and/or injection data
                    of a particular wellbore.

        Sample Link
       -------------
        www.kaggle.com/dataset/5c4a19ebeb017a77adadce8c460e2a8ca1e6e84719f1022ba317328be9bdd267

    """
    import pandas as pd
    import sys

    # Clean, transform, and sort the data
    df = create_sorted_df(data)

    # Check the validity of the entered wellbore name
    if not isinstance(well_name, str):
        sys.exit(f"Argument 'well_name' should be passed as a string, received '{type(well_name)}'")
    elif well_name.upper() not in df['Wellbore name'].unique():
        sys.exit(f"Invalid input, please enter a valid Wellbore name.\nValid names:{df['Wellbore name'].unique()}")
    else:
        well_name = well_name.upper()

    # Querying the data of the entered wellbore name
    data_well = df[df['Wellbore name'] == well_name].drop(columns=['cum_oil', 'cum_gas', 'cum_water']).reset_index(drop=True)

    return data_well

In [ ]:
def analyze_wellbore(data, well_name):
    """
    Creates a grid made up of four (04) to six (06) subplots that allow us to analyze a particular wellbore including:

     - The annual Oil & Gas production (barplots)
     - The monthly Oil & Gas production (heatmaps)
     - The total operating time (lineplot)
     - The total fluid injection (barplot)

     Note: This function does not handle injection wells


        Arguments
       -----------
        data (pd.DataFrame) : Production dataset. The passed data frame should be in the appropriate format.
        
        well_name (str) : Specifies the name of the wellbore you want to analyze.


        Sample Link
       -------------
        www.kaggle.com/dataset/5c4a19ebeb017a77adadce8c460e2a8ca1e6e84719f1022ba317328be9bdd267

    """

    import matplotlib.pyplot as plt
    import matplotlib
    import seaborn as sns
    import pandas as pd
    import math
    import sys


    # Querying the data of the entered wellbore name
    data_well = create_well_df(data, well_name)

    # Creating sub data frames
    data_year = data_well.groupby('Year')[['On Stream', 'Oil', 'Gas', 'Water', 'GI', 'WI']].sum()
    data_month_oil = data_well.pivot_table('Oil', 'Month', 'Year')
    data_month_gas = data_well.pivot_table('Gas', 'Month', 'Year')

    data_month_oil.fillna(0, inplace=True)
    data_month_gas.fillna(0, inplace=True)

    # Setting default properties
    sns.set_style('darkgrid')
    matplotlib.rcParams['font.size'] = 14
    matplotlib.rcParams['figure.facecolor'] = '#00000000'

    # To avoid getting decimal values on the X-axis while plotting the annual production
    xint = range(math.floor(min(data_year.index)), math.ceil(max(data_year.index)+1))

    # Read this carefully and make sure you understand what's going on
    col = 3 if data_year.WI.sum() > 0 or data_year.GI.sum() > 0 else 2
    fig_size = (18, 9.5) if data_year.WI.sum() > 0 or data_year.GI.sum() > 0 else (15, 11)


    if data_year.Oil.sum() > 0 or data_year.Gas.sum() > 0 or data_year.Water.sum() > 0:

        fig, axes = plt.subplots(2, col, figsize=fig_size)

        # Annual Oil Production
        axes[0, 0].bar(data_year.index, data_year.Oil, color='mediumspringgreen')
        axes[0, 0].set_xlabel('Year')
        axes[0, 0].set_ylabel('Oil production')
        axes[0, 0].set_title('Annual Oil Production (Sm$^3$)')

        # Annual Gas Production
        axes[0, 1].bar(data_year.index, data_year.Gas, color='crimson')
        axes[0, 1].set_xlabel('Year')
        axes[0, 1].set_ylabel('Gas production')
        axes[0, 1].set_title('Annual Gas Production (Sm$^3$)')

        # Monthly Oil Production
        axes[1, 0].set_title('Monthly Oil production')
        sns.heatmap(data_month_oil, cmap='Greens', ax=axes[1, 0])

        # Monthly Gas Production
        axes[1, 1].set_title('Monthly Gas production')
        sns.heatmap(data_month_gas, cmap='Reds', ax=axes[1, 1])


        if data_year.WI.sum() > 0 and data_year.GI.sum() > 0:

            # Total Operating time per year
            axes[0, 2].plot(data_year.index, data_year['On Stream'], 'o-b')
            axes[0, 2].set_xlabel('Year')
            axes[0, 2].set_ylabel('On Stream')
            axes[0, 2].set_title('Total Operating time per Year (hrs)')

            # Total fluid injection per year
            axes[1, 2].bar(data_year.index, data_year.GI, color='salmon')
            axes[1, 2].bar(data_year.index, data_year.WI, bottom=data_year.GI, color='deepskyblue')
            axes[1, 2].set_xlabel('Year')
            axes[1, 2].set_ylabel('Injected Fluid')
            axes[1, 2].legend(['Gas', 'Oil'])
            axes[1, 2].set_title('Annual Fluid Injection (Sm$^3$)')

        elif data_year.WI.sum() > 0 and data_year.GI.sum() == 0:

            # Total Operating time per year
            axes[0, 2].plot(data_year.index, data_year['On Stream'], 'o-b')
            axes[0, 2].set_xlabel('Year')
            axes[0, 2].set_ylabel('On Stream')
            axes[0, 2].set_title('Total Operating time per Year (hrs)')

            # Total Water injection per year
            axes[1, 2].bar(data_year.index, data_year.WI, color='deepskyblue')
            axes[1, 2].set_xlabel('Year')
            axes[1, 2].set_ylabel('Injected Water')
            axes[1, 2].set_title('Annual Water Injection (Sm$^3$)')

        elif data_year.WI.sum() == 0 and data_year.GI.sum() > 0:

            # Total Operating time per year
            axes[0, 2].plot(data_year.index, data_year['On Stream'], 'o-b')
            axes[0, 2].set_xlabel('Year')
            axes[0, 2].set_ylabel('On Stream')
            axes[0, 2].set_title('Total Operating time per Year (hrs)')

            # Total Gas injection per year
            axes[1, 2].bar(data_year.index, data_year.GI, color='crimson')
            axes[1, 2].set_xlabel('Year')
            axes[1, 2].set_ylabel('Injected Gas')
            axes[1, 2].set_title('Annual Gas Injection (Sm$^3$)')

        else:
            axes[0, 0].xaxis.set_ticks(xint)
            axes[0, 1].xaxis.set_ticks(xint)
            
        fig.suptitle(f"Analysis of the wellbore {well_name.upper()}")
        plt.tight_layout(pad=2)

In [ ]:
df = pd.read_excel('../input/volve-production-data/Volve production data.xlsx', sheet_name='Monthly Production Data')

In [ ]:
# Clean monthly production data in case you need it.
clean_df = cleanse_data(df)
clean_df.to_csv('volve-monthly-production-data-cleaned.csv')

In [ ]:
analyze_wellbore(df, '15/9-F-5')

In [ ]:
analyze_wellbore(df, '15/9-F-12')

In [ ]:
analyze_wellbore(df, '15/9-f-14')